In [21]:
import time

with open("./../data/test_performance_input.txt", "r") as f:
    test_input = [input_str.strip() for input_str in f.read().split("###")]
    test_input = [text for text in test_input if text != ""]

len(test_input)

1

In [23]:
import httpx
import json

test_results = []

def run_test(input: str):
    start = time.time()
    result = httpx.post("http://localhost:8000/processText", json={
        "text": input
    }, timeout=None)
    end = time.time()
    duration = end - start
    test_results.append({"duration": duration, "input": input, "result": result.json()})
    print("\033[4mTest result:\033[0m")
    print(f"\033[1mDuration:\033[0m {duration}s")
    print(f"\033[1mInput:\033[0m \"{input}\"")
    for term in result.json()["terms"]:
        print(f"- {term['text'] if term['normalization'] == '' else term['normalization']}")
        for definition in term["definitions"]:
            prefix = ""
            if definition["partial"]:
                prefix += "⚙️"
            if not definition["verified"]:
                prefix += "✨"
            else:
                prefix += "☑️"
            print(f"\t- {prefix} {definition['text']}")
    print()

for text in test_input:
    try:
        run_test(text)
    except:
        print(f"Test failed: {text}")

with open("./../data/test_performance_output.json", "w") as f:
    f.write(json.dumps(test_results))

Test result:
Duration: 1.2993950843811035s
Input: "Baustelle an Bahnübergang"
- Baustelle
- Bahnübergang
	- ☑️ höhengleiche (in der Ebene der Schienenoberkante [SO] liegende) Kreuzung zwischen Eisenbahngleisen und Straßen, Wegen und Plätzen, wobei dem Schienenweg der Vorrang zusteht. Man unterscheidet zwischen technisch gesicherten und technisch nicht gesicherten Bahnübergängen. Die technische Sicherung kann bestehen aus zugbetätigtem Haltlicht, aus zugbetätigtem und fernüberwachtem Haltlicht mit selbsttätigen Halbschranken oder aus hand- oder fernbedienten Vollschranken mit oder ohne Verkehrssignalanlage.
	- ☑️ höhengleiche (in der Ebene der Schienenoberkante [SO] liegende) Kreuzung zwischen Eisenbahngleisen und Straßen, Wegen und Plätzen, wobei dem Schienenweg der Vorrang zusteht. Man unterscheidet zwischen technisch gesicherten und technisch nicht gesicherten Bahnübergängen. Die technische Sicherung kann bestehen aus zugbetätigtem Haltlicht, aus zugbetätigtem und fernüberwachtem Hal